In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from weather import Weather
import warnings
warnings.filterwarnings('ignore')

In [62]:
data = pd.read_csv("input_data.csv")
keyword = input('Enter keyword: ')
print('Here are some items that match your keyword:')

results = []

for i in range(len(data['Description'])):
    if keyword in data['Description'][i]:
        results.append((i, data['Description'][i]))
    if len(results) == 6:
        break

for result in results:
    print(result[0], ' ', result[1])

print('\n')
print('Please pick an item so we can find more recommendations.')
reference_item = int(input('Enter ID:'))
a = data['Description'][reference_item] + "outfit worn before"
print(a)


Enter keyword: sweater
Here are some items that match your keyword:
14   Forever 21 purple furry sweater: The warmest fuzziest sweater that looks best with a black skirt or denim overalls
40   Wilson college merino wool black sweater: A baggy black sweater for a nice fall day  and show off Wilson pride
53   Cheque black and grey turtleneck: The shortsleeved turtle neck sweater is perfect for providing some neck warmth from cold Boston winds
56   Oxford teal lambwool sweater: A very warm snug lambwool sweater to keep you cozy in summer with blue jeans
57   Oxford purple lambwool sweater: A very warm snug lambwool sweater to keep you cozy in summer with blue jeans
81   Forever21 black skirt: A black miniskirt which goes great with tshirts or sweaters


Please pick an item so we can find more recommendations.
Enter ID:12
Forever 21 black dress with mesh neck: It's perfect for professional classy black look. The neck provides an elegant feature to a simple black dress.outfit worn before


In [ ]:
data = pd.read_csv("input_data.csv")
weather = Weather(unit='c')

current_location = 'Sydney'
location = weather.lookup_by_location(current_location)
condition = location.condition().text()

forecasts = location.forecast()
weather_lo = int(forecasts[0].low())
weather_hi = int(forecasts[0].high())

if weather_hi >= 18 and weather_lo <= 7:
    data['Description'][reference_item] += ' fall spring layer'
    weather_condition = 'Fall/Spring'
elif weather_hi > 20:
    data['Description'][reference_item] += ' hot sunny summer shorts skirt'
    weather_condition = 'Hot'
elif weather_lo < 5:
    data['Description'][reference_item] += ' cold warm cozy sweater'
    weather_condition = 'Cold'
    
message = '{} weather detected in {}. Recommendations are being tailored accordingly.'
print(message.format(weather_condition, current_location))

In [ ]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(data['Description'])
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
results = {}

In [ ]:
for idx, row in data.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], data['ID'][i]) for i in similar_indices]

    # First item is the item itself, so remove it.
    # Each dictionary entry is like: [(1,2), (3,4)], with each tuple being (score, item_id)
    results[row['ID']] = similar_items[1:]

In [49]:
def item(id):
    return data.loc[data['ID'] == id]['Description'].tolist()[0].split(' - ')[0]

def recommend(item_id, num):
    print("Recommending " + str(num) + " clothes similar to " + item(item_id).split(':')[0] + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print(item(rec[1]) + " (score: " + str(round(rec[0], 5)) + ")")
recommend(item_id=reference_item, num=9)



Recommending 9 clothes similar to Forever 21 purple furry sweater...
-------
Forever 21 maroon crop tshirt: Looks really cute with black shorts or a black skirt on a hot sunny day (score: 0.15044)
Gap purple fine merino wool sweater: Light purple sweater that looks really classy with blue denim or grey denim (score: 0.12406)
Uniqlo rayon olive green blouse: This sleeveless cotton blouse is very elegant. It's perfect for dressing up or down for casual or work . Looks best with blue jeans or black denim.Can also be paired with a black skirt. It's perfect for a warm sunny summer day. Black Tommy Hilfiger shoes are perfect with this (score: 0.09182)
Forever 21 black corduroy shorts: Best for showing off legs (score: 0.0863)
Oxford purple lambwool sweater: A very warm snug lambwool sweater to keep you cozy in summer with blue jeans (score: 0.08511)
Forever 21 green tank: chiiffon green tank is very comfortable and look so hot with the black shorts or the Jcrew shorts (score: 0.0695)
DKNY To